🎯 CONTEXTO DEL PROYECTO

¿Dónde estamos ahora?
Hemos completado exitosamente la Fase 1|2: EDA y Limpieza. Tenemos un dataset limpio con 1,470 empleados y 30 variables listas para analizar.

¿Cuál es el objetivo del negocio?

ABC Corporation quiere:

✅ Reducir la rotación de empleados (attrition)

✅ Mejorar la satisfacción laboral

✅ Identificar factores clave que influyen en ambos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_csv('../data/processed/hr_processed.csv')

In [ ]:
df = pd.read_csv('../data/processed/hr_processed.csv')
if 'id' in df.columns:
    df.set_index('id', inplace=True)

print(f"✅ Dataset cargado: {df.shape[0]} empleados × {df.shape[1]} variables")

**Objetivo:** Identificar factores de riesgo y segmentos críticos para reducir la rotación del 16.1% actual

## 📊 PASO 1: DIAGNÓSTICO RÁPIDO

Antes de sumergirme en gráficos, necesito entender la magnitud del problema.

In [ ]:
# KPIs principales
total_empleados = len(df)
se_van = (df['attrition'] == 'Yes').sum()
tasa_rotacion = (df['attrition'] == 'Yes').mean() * 100

print("=" * 70)
print("DIAGNÓSTICO RÁPIDO - SITUACIÓN ACTUAL")
print("=" * 70)
print(f"\n📊 Total empleados: {total_empleados:,}")
print(f"📈 Se van: {se_van} ({tasa_rotacion:.1f}%)")
print(f"📉 Se quedan: {total_empleados - se_van} ({100-tasa_rotacion:.1f}%)")

# Benchmark
benchmark_tech = 13.5  # Promedio industria tech
diferencia = tasa_rotacion - benchmark_tech

if tasa_rotacion > 20:
    nivel = "🔴 CRÍTICO"
elif tasa_rotacion > 15:
    nivel = "🟠 ALTO"
elif tasa_rotacion > 10:
    nivel = "🟡 MODERADO"
else:
    nivel = "🟢 BAJO"

print(f"\n{nivel}")
print(f"Benchmark industria: {benchmark_tech}%")
print(f"Diferencia: {diferencia:+.1f} puntos porcentuales")

# Perfil rápido: ¿Quiénes se van?
print(f"\n💡 PERFIL DE QUIENES SE VAN vs QUIENES SE QUEDAN:\n")

comparativa = df.groupby('attrition').agg({
    'age': 'mean',
    'monthly_income': 'mean',
    'years_at_company': 'mean'
}).T

for metric in comparativa.index:
    val_quedan = comparativa.loc[metric, 'No']
    val_van = comparativa.loc[metric, 'Yes']
    diff = val_quedan - val_van
    pct = (diff / val_van) * 100
    
    print(f"   {metric.replace('_', ' ').title()}:")
    print(f"      Quedan: {val_quedan:,.1f}")
    print(f"      Se van: {val_van:,.1f}")
    print(f"      → Diferencia: {diff:+,.1f} ({pct:+.1f}%)\n")

## 🎯 PASO 2: IDENTIFICAR TOP FACTORES DE RIESGO

No todos los factores son igual de importantes. Voy a calcular el **riesgo relativo** de cada uno.

In [ ]:
def calcular_riesgo_relativo(condicion_str):
    """
    Calcula cuántas veces más probable es que alguien se vaya 
    si cumple vs no cumple una condición.
    """
    con_factor = df.query(condicion_str)
    sin_factor = df.query(f'not ({condicion_str})')
    
    tasa_con = (con_factor['attrition'] == 'Yes').mean() * 100
    tasa_sin = (sin_factor['attrition'] == 'Yes').mean() * 100
    
    riesgo_relativo = tasa_con / tasa_sin if tasa_sin > 0 else 0
    
    return tasa_con, riesgo_relativo, len(con_factor)

# Evaluar factores candidatos
factores = {
    'Overtime (hace horas extra)': 'over_time == "Yes"',
    'Empleado nuevo (<2 años)': 'years_at_company < 2',
    'Baja satisfacción laboral': 'job_satisfaction in ["Nada satisfecho", "Insatisfecho"]',
    'Lejos de casa (>15km)': 'distance_from_home > 15',
    'Mal balance vida-trabajo': 'work_life_balance in ["Nada satisfecho", "Insatisfecho"]',
    'Salario bajo (< mediana)': 'monthly_income < monthly_income.median()',
    'Sin stock options': 'stock_option_level == 0',
    'Bajo involucramiento': 'job_involvement == "Insatisfecho"'
}

resultados = {}
for nombre, condicion in factores.items():
    tasa, rr, count = calcular_riesgo_relativo(condicion)
    resultados[nombre] = {'tasa': tasa, 'rr': rr, 'count': count}

# Ordenar por riesgo relativo
resultados_df = pd.DataFrame(resultados).T.sort_values('rr', ascending=False)

print("=" * 80)
print("TOP FACTORES DE RIESGO (ordenados por impacto)")
print("=" * 80)
print(f"\n{'Factor':<40} {'% Rotación':<12} {'Riesgo':<10} {'Afectados'}")
print("-" * 80)

for i, (factor, row) in enumerate(resultados_df.head(5).iterrows(), 1):
    print(f"{i}. {factor:<37} {row['tasa']:>6.1f}%      {row['rr']:>5.2f}x     {row['count']:>4.0f}")

print(f"\n💡 INTERPRETACIÓN:")
print(f"   'Riesgo' = Cuántas veces más probable es que se vayan vs quienes NO tienen ese factor")
print(f"   Ej: {resultados_df.index[0]} tiene {resultados_df.iloc[0]['rr']:.2f}x más riesgo")

## 🎲 PASO 3: MODELO DE SEGMENTACIÓN DE RIESGO

Voy a crear un score de riesgo que combine múltiples factores para identificar empleados en peligro.

In [ ]:
# Crear score de riesgo ponderado
df['risk_score'] = 0

# Ponderar por riesgo relativo (redondeado)
df['risk_score'] += (df['over_time'] == 'Yes').astype(int) * 3  # RR: ~3x
df['risk_score'] += (df['years_at_company'] < 2).astype(int) * 3  # RR: ~2.7x
df['risk_score'] += (df['job_satisfaction'].isin(['Nada satisfecho', 'Insatisfecho'])).astype(int) * 2
df['risk_score'] += (df['work_life_balance'].isin(['Nada satisfecho', 'Insatisfecho'])).astype(int) * 2
df['risk_score'] += (df['distance_from_home'] > 15).astype(int) * 1
df['risk_score'] += (df['monthly_income'] < df['monthly_income'].median()).astype(int) * 1

# Clasificar en categorías
df['risk_category'] = pd.cut(
    df['risk_score'], 
    bins=[-1, 2, 5, 15], 
    labels=['BAJO', 'MEDIO', 'ALTO']
)

print("=" * 70)
print("MODELO DE SEGMENTACIÓN DE RIESGO")
print("=" * 70)

# Distribución
print(f"\n📊 DISTRIBUCIÓN:")
for cat in ['ALTO', 'MEDIO', 'BAJO']:
    count = (df['risk_category'] == cat).sum()
    pct = count / len(df) * 100
    print(f"   {cat:6}: {count:4} empleados ({pct:5.1f}%)")

# Validación: ¿funciona el modelo?
print(f"\n✅ VALIDACIÓN DEL MODELO (¿predice bien?):\n")

for cat in ['BAJO', 'MEDIO', 'ALTO']:
    grupo = df[df['risk_category'] == cat]
    tasa_real = (grupo['attrition'] == 'Yes').mean() * 100
    total = len(grupo)
    se_van = (grupo['attrition'] == 'Yes').sum()
    
    print(f"   Riesgo {cat:6}: {tasa_real:5.1f}% rotación real ({se_van}/{total})")

# Empleados actuales en riesgo ALTO
actuales_alto_riesgo = df[(df['attrition'] == 'No') & (df['risk_category'] == 'ALTO')]
perdida_estimada = int(len(actuales_alto_riesgo) * 0.40)  # 40% de ellos se irán

print(f"\n🚨 ALERTA CRÍTICA:")
print(f"   {len(actuales_alto_riesgo)} empleados ACTUALES en ALTO RIESGO")
print(f"   Estimación: ~{perdida_estimada} se irán si no actuamos")

# Costo estimado
costo_por_empleado = df['monthly_income'].mean() * 12 * 1.75  # 1.75x salario anual
costo_total = perdida_estimada * costo_por_empleado
print(f"   Costo estimado: ${costo_total/1000000:.1f}M USD")

## 🏢 PASO 4: ANÁLISIS POR DEPARTAMENTO

¿Todos los departamentos tienen el mismo problema?

In [ ]:
# Análisis completo por departamento
dept_analysis = df.groupby('department').agg({
    'attrition': lambda x: (x == 'Yes').mean() * 100,
    'monthly_income': 'mean',
    'over_time': lambda x: (x == 'Yes').mean() * 100,
    'job_satisfaction': lambda x: (x.isin(['Satisfecho', 'Muy satisfecho'])).mean() * 100,
    'years_at_company': 'mean'
}).round(1)

dept_analysis.columns = ['% Rotación', 'Salario Prom', '% Overtime', '% Satisfechos', 'Antigüedad']
dept_analysis['Total'] = df['department'].value_counts()
dept_analysis = dept_analysis.sort_values('% Rotación', ascending=False)

print("=" * 95)
print("ANÁLISIS POR DEPARTAMENTO")
print("=" * 95)
print(f"\n{'Departamento':<30} {'Total':<8} {'Rotación':<12} {'Salario':<12} {'Overtime':<12} {'Satisf.'}")
print("-" * 95)

for dept, row in dept_analysis.iterrows():
    print(f"{dept:<30} {row['Total']:<8.0f} {row['% Rotación']:>6.1f}%      ${row['Salario Prom']:>8,.0f}   {row['% Overtime']:>6.1f}%      {row['% Satisfechos']:>5.1f}%")

# Departamento crítico
dept_critico = dept_analysis.index[0]
print(f"\n🎯 DEPARTAMENTO CRÍTICO: {dept_critico}")
print(f"   → Rotación: {dept_analysis.loc[dept_critico, '% Rotación']:.1f}%")
print(f"   → Afecta a {dept_analysis.loc[dept_critico, 'Total']:.0f} empleados")
print(f"   → Requiere intervención INMEDIATA")

## 💰 PASO 5: ANÁLISIS DE COMPENSACIÓN

¿El dinero importa? (spoiler: sí)

In [ ]:
print("=" * 70)
print("ANÁLISIS DE COMPENSACIÓN")
print("=" * 70)

# Brecha salarial
sal_quedan = df[df['attrition'] == 'No']['monthly_income'].mean()
sal_van = df[df['attrition'] == 'Yes']['monthly_income'].mean()
brecha = sal_quedan - sal_van
brecha_pct = (brecha / sal_van) * 100

print(f"\n💰 BRECHA SALARIAL:")
print(f"   Quedan: ${sal_quedan:,.0f}/mes")
print(f"   Se van: ${sal_van:,.0f}/mes")
print(f"   Brecha: ${brecha:,.0f} ({brecha_pct:.1f}% menos)")

# Stock options
print(f"\n🎁 IMPACTO DE STOCK OPTIONS:")
for nivel in [0, 1, 2, 3]:
    grupo = df[df['stock_option_level'] == nivel]
    tasa = (grupo['attrition'] == 'Yes').mean() * 100
    count = len(grupo)
    print(f"   Nivel {nivel}: {tasa:5.1f}% rotación ({count} empleados)")

# ROI de dar stock options
sin_stock = df[df['stock_option_level'] == 0]
con_stock = df[df['stock_option_level'] > 0]
reduccion = ((sin_stock['attrition'] == 'Yes').mean() - (con_stock['attrition'] == 'Yes').mean()) * 100

print(f"\n📊 ROI DE STOCK OPTIONS:")
print(f"   Reducción de rotación: {reduccion:.1f} puntos porcentuales")
print(f"   Empleados sin stock: {len(sin_stock)}")
print(f"   → Dar stock a todos podría evitar ~{int(len(sin_stock) * reduccion / 100)} salidas")